In [8]:
import sqlite3
from langchain_core.messages import SystemMessage, HumanMessage, RemoveMessage
from langchain_openai import ChatOpenAI
from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.graph import MessagesState, StateGraph, START, END
from IPython.display import display, Image

In [5]:
llm = ChatOpenAI(model="gpt-4o", temperature=0)


class State(MessagesState):
    summary: str


def call_model(state: State):
    summary = state.get("summary", "")

    if summary:
        system_message = SystemMessage(content=f"Summary of conversation earlier: {summary}")
        messages = [system_message] + state["messages"]
    else:
        messages = state["messages"]

    response = llm.invoke(messages)
    return {"messages": [response]}


def summarize_conversation(state: State):
    summary = state.get("summary", "")

    if summary:
        summary_message = HumanMessage(content=(
            f"This is summary of the conversation to date: {summary}\n\n"
            "Extend the summary by taking into account the new messages above:"
        ))
    else:
        summary_message = HumanMessage(content="Create a summary of the conversation above:")

    messages = state["messages"] + [summary_message]
    response = llm.invoke(messages)

    delete_messages = [RemoveMessage(id=m.id) for m in state["messages"][:-2]]
    return {"summary": response.content, "messages": delete_messages}


def should_continue(state: State):
    messages = state["messages"]
    if len(messages) > 6:
        return "summarize_conversation"
    return END

In [11]:
workflow = StateGraph(State)

workflow.add_node("conversation", call_model)
workflow.add_node(summarize_conversation)

workflow.add_edge(START, "conversation")
workflow.add_conditional_edges("conversation", should_continue)
workflow.add_edge("summarize_conversation", END)

conn = sqlite3.connect("state_db/example.db", check_same_thread=False)
memory = SqliteSaver(conn)
graph = workflow.compile(checkpointer=memory)

In [12]:
config = {"configurable": {"thread_id": "1"}}

input_message = HumanMessage(content="hi! I'm Lance")
output = graph.invoke({"messages": [input_message]}, config) 
for m in output['messages'][-1:]:
    m.pretty_print()

input_message = HumanMessage(content="what's my name?")
output = graph.invoke({"messages": [input_message]}, config) 
for m in output['messages'][-1:]:
    m.pretty_print()

input_message = HumanMessage(content="i like the 49ers!")
output = graph.invoke({"messages": [input_message]}, config) 
for m in output['messages'][-1:]:
    m.pretty_print()

================================== Ai Message ==================================

Hi again, Lance! It's great to hear from you. Since you like the 49ers, would you like to dive into a specific topic about the team? Or is there something else you'd like to chat about? Let me know!
================================== Ai Message ==================================

Your name is Lance! 😊 Did I get it right?
================================== Ai Message ==================================

That's awesome, Lance! The 49ers are such a legendary team with a rich history. Do you have a favorite player, past or present? Or maybe a favorite game or moment? Let’s talk Niners! 🏈


In [13]:
config = {"configurable": {"thread_id": "1"}}
graph_state = graph.get_state(config)
graph_state

StateSnapshot(values={'messages': [HumanMessage(content="hi! I'm Lance", additional_kwargs={}, response_metadata={}, id='425c043e-d52a-4ec1-8b26-8db8579012b8'), AIMessage(content="Hi again, Lance! It's great to hear from you. Since you like the 49ers, would you like to dive into a specific topic about the team? Or is there something else you'd like to chat about? Let me know!", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 49, 'prompt_tokens': 337, 'total_tokens': 386, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-11-20', 'system_fingerprint': 'fp_ee1d74bde0', 'id': 'chatcmpl-BrMESykvBSLJ8QfRWYa87lug7CP0F', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--555ac8b8-326a-48ae-9cd6-ea7fbc20d906-0', usage_metadata={'input_tokens': 337